In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import env

In [2]:
url = env.get_db_url('employees')

sql = 'SELECT * FROM employees'
employees_df = pd.read_sql(sql, url)

sql = 'SELECT * FROM salaries'
salaries_df = pd.read_sql(sql, url)

In [3]:
sql = 'SELECT * FROM dept_emp'
dept_emp_df = pd.read_sql(sql, url)

In [4]:
emp = employees_df
sal = salaries_df
de = dept_emp_df

In [5]:
df = pd.merge(emp, de, on='emp_no')

In [6]:
df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,dept_no,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,d005,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,d007,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,d004,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,d004,1986-12-01,9999-01-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,d003,1989-09-12,9999-01-01


Write a query that returns all employees, their department number, their start date, their end date, and a new column 'is_current_employee' that is a 1 if the employee is still with the company and 0 if not.

In [7]:
df2 = (
        pd.merge(emp, 
                de.groupby('emp_no').to_date.max().reset_index(),
                on='emp_no'
                )
                .rename(columns={'to_date': 'end_date'})
                [['emp_no', 'hire_date', 'end_date']]
      )
df2['is_current_employee'] = np.where(df2.end_date > pd.to_datetime('today'), 1, 0)
df2 = pd.merge(de[['emp_no', 'dept_no']], df2, how='right', on='emp_no')

df2

,emp_no,dept_no,hire_date,end_date,is_current_employee
0,10001,d005,1986-06-26,9999-01-01,1
1,10002,d007,1985-11-21,9999-01-01,1
2,10003,d004,1986-08-28,9999-01-01,1
3,10004,d004,1986-12-01,9999-01-01,1
4,10005,d003,1989-09-12,9999-01-01,1
...,...,...,...,...,...
331598,499995,d004,1993-01-12,9999-01-01,1
331599,499996,d004,1990-09-27,9999-01-01,1
331600,499997,d005,1986-04-21,9999-01-01,1
331601,499998,d002,1993-10-13,9999-01-01,1


Write a query that returns all employee names 
(previous and current), and a new column 'alpha_group' 
that returns 'A-H', 'I-Q', or 'R-Z' depending on the 
first letter of their last name.

In [8]:
def get_alpha_group(name):
    if name[0] < 'I':
        return 'A-H'
    elif name[0] < 'R':
        return 'I-Q'
    else:
        return 'R-Z'
df2 = employees_df[['last_name', 'first_name']]
df2['alpha_group'] = df2.last_name.apply(get_alpha_group)

df2

,last_name,first_name,alpha_group
0,Facello,Georgi,A-H
1,Simmel,Bezalel,R-Z
2,Bamford,Parto,A-H
3,Koblick,Chirstian,I-Q
4,Maliniak,Kyoichi,I-Q
...,...,...,...
300019,Lichtner,Dekang,I-Q
300020,Baaz,Zito,A-H
300021,Lenart,Berhard,I-Q
300022,Breugel,Patricia,A-H


How many employees (current or previous) were born in each decade?

In [9]:
def get_decade(birthdate):
    if str(birthdate).startswith('195'):
        return '50s'
    elif str(birthdate).startswith('196'):
        return '60s'
    else:
        return 'other'
        
emp.birth_date.apply(get_decade).value_counts()

50s    182886
60s    117138
Name: birth_date, dtype: int64